https://incredible.ai/machine-learning/2019/02/10/Q-Learning/  
https://lee-jaewon.github.io/reinforcement_learning/RL_intro_3/

In [ ]:
env = gym.make('MountainCar-v0')
env.observation_space.high  # array([0.6 , 0.07], dtype=float32)
env.observation_space.low   # array([-1.2 , -0.07], dtype=float32)

array([-1.2 , -0.07], dtype=float32)

In [ ]:
env = gym.make('MountainCar-v0')
n_state = (env.observation_space.high - env.observation_space.low) * np.array([10, 100])
n_state = np.round(n_state, 0).astype(int) + 1

Q = np.random.uniform(-1, 1, size=(n_state[0], n_state[1], env.action_space.n))
print('Q shape:', Q.shape)
print('Q Table')
print(Q[1:2])

Q shape: (19, 15, 3)
Q Table
[[[ 0.5598881   0.55363559  0.56942304]
  [-0.46180549  0.63332862  0.42495505]
  [ 0.12902945 -0.09786553  0.22399982]
  [ 0.14215785  0.66569276 -0.4691401 ]
  [ 0.34137618  0.3086423   0.50699602]
  [ 0.02582433 -0.21722055  0.70917064]
  [-0.78174958 -0.44021225 -0.72401939]
  [-0.91302107 -0.89274312 -0.0651827 ]
  [-0.51054098  0.17135171  0.14322992]
  [-0.33646288 -0.72559437  0.9739868 ]
  [ 0.68338796  0.51602966  0.7471864 ]
  [-0.42211721  0.76165745  0.78417915]
  [-0.43535923 -0.54087993 -0.13789801]
  [ 0.26094848 -0.45425332  0.75604422]
  [ 0.76369095  0.91039326  0.85233833]]]


In [ ]:
from tqdm import tqdm_notebook
def discretize(env, state):
    state = (state - env.observation_space.low) * np.array([10, 100])
    state = np.round(state, 0).astype(int)
    return state

def train(env, Q, epochs=10000, lr=0.1, gamma=0.9, epsilon=0.9):
    reduction = epsilon/epochs
    action_n = env.action_space.n

    rewards = list()

    for epoch in tqdm_notebook(range(epochs)):
        state = env.reset()
        state = discretize(env, state)

        done = False
        _tot_reward = 0
        _tot_rand_action = 0
        _tot_q_action = 0
        _max_pos = 0

        while not done:

            # Calculate next action
            if np.random.random() < 1 - epsilon:
                action = np.argmax(Q[state[0], state[1]])
                _tot_q_action += 1
            else:
                action = np.random.randint(0, action_n)
                _tot_rand_action += 1

            # Step!
            next_state, reward, done, info = env.step(action)
            next_state_apx = discretize(env, next_state)

            # Terminal Update
            if done and next_state[0] >= 0.5:
                Q[next_state_apx[0], next_state_apx[1], action] = reward
            else:
                delta = lr * (reward + gamma * np.max(Q[next_state_apx[0], next_state_apx[1]]) -
                              Q[state[0], state[1], action])
                Q[state[0], state[1], action] += delta

            state = next_state_apx
            _tot_reward += reward

        # Decay Epsilon
        if epsilon > 0:
            epsilon -= reduction
            epsilon = round(epsilon, 4)

        # Track Rewards
        rewards.append(_tot_reward)

        # Log
        if epoch%100 == 0:
            print(f'\repoch:{epoch} | tot reward:{_tot_reward} | epsilon:{epsilon} | '
                  f'rand action:{_tot_rand_action} | Q action:{_tot_q_action}')

train(env, Q)

<ipython-input-15-f57416b1c6e8>:13: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for epoch in tqdm_notebook(range(epochs)):


  0%|          | 0/10000 [00:00<?, ?it/s]

epoch:0 | tot reward:-200.0 | epsilon:0.8999 | rand action:185 | Q action:15
epoch:100 | tot reward:-200.0 | epsilon:0.8899 | rand action:176 | Q action:24
epoch:200 | tot reward:-200.0 | epsilon:0.8799 | rand action:178 | Q action:22
epoch:300 | tot reward:-200.0 | epsilon:0.8699 | rand action:169 | Q action:31
epoch:400 | tot reward:-200.0 | epsilon:0.8599 | rand action:173 | Q action:27
epoch:500 | tot reward:-200.0 | epsilon:0.8499 | rand action:168 | Q action:32
epoch:600 | tot reward:-200.0 | epsilon:0.8399 | rand action:168 | Q action:32
epoch:700 | tot reward:-200.0 | epsilon:0.8299 | rand action:166 | Q action:34
epoch:800 | tot reward:-200.0 | epsilon:0.8199 | rand action:161 | Q action:39
epoch:900 | tot reward:-200.0 | epsilon:0.8099 | rand action:169 | Q action:31
epoch:1000 | tot reward:-200.0 | epsilon:0.7999 | rand action:163 | Q action:37
epoch:1100 | tot reward:-200.0 | epsilon:0.7899 | rand action:146 | Q action:54
epoch:1200 | tot reward:-200.0 | epsilon:0.7799 | ra

In [ ]:
env = gym.make('MountainCar-v0')
state = env.reset()
state = discretize(env, state)

env.render()
input()

while True:
    env.render()
    action = np.argmax(Q[state[0], state[1]])
    state, reward, done, info = env.step(action)
    state = discretize(env, state)

    print(f'\rstate:{state} | reward:{reward} | done:{done} | info:{info}')

    if done:
        break

/usr/local/lib/python3.10/dist-packages/gym/core.py:49: DeprecationWarning: WARN: You are calling render method, but you didn't specified the argument render_mode at environment initialization. To maintain backward compatibility, the environment will render in human mode.
If you want to render in human mode, initialize the environment in this way: gym.make('EnvName', render_mode='human') and don't call the render method.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(


0.9
state:[8 7] | reward:-1.0 | done:False | info:{}
state:[8 7] | reward:-1.0 | done:False | info:{}
state:[8 6] | reward:-1.0 | done:False | info:{}
state:[8 6] | reward:-1.0 | done:False | info:{}
state:[8 6] | reward:-1.0 | done:False | info:{}
state:[7 6] | reward:-1.0 | done:False | info:{}
state:[7 6] | reward:-1.0 | done:False | info:{}
state:[7 6] | reward:-1.0 | done:False | info:{}
state:[7 6] | reward:-1.0 | done:False | info:{}
state:[7 5] | reward:-1.0 | done:False | info:{}
state:[7 5] | reward:-1.0 | done:False | info:{}
state:[7 5] | reward:-1.0 | done:False | info:{}
state:[6 5] | reward:-1.0 | done:False | info:{}
state:[6 5] | reward:-1.0 | done:False | info:{}
state:[6 5] | reward:-1.0 | done:False | info:{}
state:[6 5] | reward:-1.0 | done:False | info:{}
state:[6 5] | reward:-1.0 | done:False | info:{}
state:[5 5] | reward:-1.0 | done:False | info:{}
state:[5 5] | reward:-1.0 | done:False | info:{}
state:[5 5] | reward:-1.0 | done:False | info:{}
state:[5 5] | re